In [1]:
import pandas as pd

In [2]:
# FUNCTION DEFINITION: function to create DataFrameMatrix with define row_name and col_name 
def matrix_maker_function(dataframe,row_name,col_name,value_name):
    df = dataframe.filter([row_name,col_name,value_name], axis=1)
    matrix_df = pd.pivot_table(df,index=[row_name],values=[value_name],columns=[col_name], aggfunc='sum')             
    matrix_df = pd.DataFrame(matrix_df.to_records())                                
    matrix_df.columns = [hdr.replace("('", "").replace("', ", "").replace(value_name, "").replace(")", "") \
                     for hdr in matrix_df.columns]
    return matrix_df

In [3]:
def filled_matrix_function(dataframe,row_name,col_name,value_name):
    df = dataframe.filter([row_name,col_name,value_name], axis=1)
    matrix_df = pd.pivot_table(df,index=[row_name],values=[value_name],columns=[col_name], aggfunc='sum')             
    matrix_df = pd.DataFrame(matrix_df.to_records())                                
    matrix_df.columns = [hdr.replace("('", "").replace("', ", "").replace(value_name, "").replace(")", "") \
                     for hdr in matrix_df.columns]
    matrix_df = matrix_df.set_index(row_name).transpose().round(2)
    matrix_df.fillna(method='bfill', inplace=True)
    matrix_df.fillna(method='ffill', inplace=True)
    matrix_df = matrix_df.transpose().reset_index()
    matrix_df = matrix_df[matrix_df[row_name].isin(HappyCountries[row_name])].set_index(row_name)
    matrix_df.drop(matrix_df.columns[[0, 1, 13, 14]], axis = 1, inplace = True)
    return matrix_df

In [4]:
pd.set_option("display.max_rows",None)

# Process and evaluate basic Happiness Index data 

We start the process with molding Happiness Index data into a matrix of countries - years to evaluate its completess and decide which years and countries are included in the data that we use for evaluation.

In [5]:
#Read main Happiness Index file - WHR20_DataForTable2.1.xls
happyindex_file = pd.read_excel("../Resources/WHR20_DataForTable2.1.xls")     
happyindex_file.replace(['Taiwan Province of China'], ['Taiwan'],inplace=True)
country_code = pd.read_csv("../Resources/countrycode1.csv")

In [6]:
country_code.head(1)

,Country name,Code,Continent
0,Afghanistan,AFG,Asia


In [7]:
# create matrix for all happiness index
Happy_Index_df = matrix_maker_function(happyindex_file ,'Country name','year','Life Ladder')
print(Happy_Index_df.shape)
Happy_Index_df.head(1)

(166, 16)


,Country name,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Afghanistan,NaN,NaN,NaN,3.72359,4.401778,4.758381,3.831719,3.782938,3.5721,3.130896,3.982855,4.220169,2.661718,2.694303,2.375092


In [8]:
# check the completeness of data
Happy_Index_df.isna().sum().head(3)

Country name      0
2005            139
2006             77
dtype: int64

HappinessIndex data is the most incomplete for 2005 - and 

In [9]:
THappy_Index_df = Happy_Index_df.set_index('Country name').transpose()
THappy_Index_df.isna().sum().head(1)

Country name
Afghanistan    3
dtype: int64

2005 and 2006 has fairly incomplete data set - thus we eliminate those two years. 
Most country have complete data, but a few has less than 10 data points, we eliminate those countries and come up with data set of 128 countries. 

In [10]:
#create the matrix of countries that we are going to study
df1 = Happy_Index_df.drop(Happy_Index_df.columns[1:3], axis=1).round(2)
df1 = df1.dropna(thresh=10).reset_index(drop=True)
print(df1.shape)
df1.to_csv("../Outputs/HappyIndex_multiyear_mx.csv", index=False)
df1.head(1)

(128, 14)


,Country name,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Afghanistan,NaN,3.72,4.4,4.76,3.83,3.78,3.57,3.13,3.98,4.22,2.66,2.69,2.38


In [11]:
#check list of rows to be deleted
outcast_df = Happy_Index_df[~Happy_Index_df['Country name'].isin(df1['Country name'])].reset_index(drop=True)
outcast_df.head(1)

,Country name,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Algeria,NaN,NaN,NaN,NaN,NaN,5.463567,5.317194,5.604596,NaN,6.354898,NaN,5.340854,5.248912,5.043086,4.744627


Create base dataframe that only has data of the years and countries that we evaluate 

In [12]:
# create a base data frame of countries that in our country list and the country codes .reset_index(drop=True)
HappyCountries = df1.filter(['Country name'], axis=1)

mapping = dict(country_code[['Country name', 'Code']].values)
HappyCountries['Code'] = country_code['Country name'].map(mapping)
HappyCountries.to_csv("../Outputs/HappyCountries.csv", index=False)
print(HappyCountries.shape)
HappyCountries.head(1)

(128, 2)


,Country name,Code
0,Afghanistan,AFG


In [13]:
# drop rows and columns that has too much NaN from original file
new_df = happyindex_file[~happyindex_file['Country name'].isin(outcast_df['Country name'])].round(2)
new_df = new_df[new_df['year'] != (2005,2006)]
new_df = new_df.drop(new_df.columns[9:26], axis=1).reset_index(drop=True)

### Create Evaluation Matrixes

Create clean matrix (with no Nan) by filling data from succeeding years (if available) with "bfill" and the completed with "ffill" for those that still incomplete. Because the operations can only be performed on columns, the dataframe need to be transpose before the operation and retranspose it afterwards.

In [14]:
HI_df = matrix_maker_function(happyindex_file ,'Country name','year','Life Ladder').set_index("Country name").transpose().round(2)
HI_df.fillna(method='bfill', inplace=True)
HI_df.fillna(method='ffill', inplace=True)
HI_df = HI_df.transpose().reset_index()
HI_df = HI_df[HI_df['Country name'].isin(HappyCountries['Country name'])].set_index("Country name")
HI_df.drop(HI_df.columns[[0, 1, 13, 14]], axis = 1, inplace = True)
HI_df

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
Country name,,,,,,,,,,,
Afghanistan,3.72,3.72,4.40,4.76,3.83,3.78,3.57,3.13,3.98,4.22,2.66
Albania,4.63,5.49,5.49,5.27,5.87,5.51,4.55,4.81,4.61,4.51,4.64
Argentina,6.07,5.96,6.42,6.44,6.78,6.47,6.58,6.67,6.70,6.43,6.04
Armenia,4.88,4.65,4.18,4.37,4.26,4.32,4.28,4.45,4.35,4.33,4.29
Australia,7.29,7.25,7.45,7.45,7.41,7.20,7.36,7.29,7.31,7.25,7.26
Austria,7.18,7.18,7.30,7.30,7.47,7.40,7.50,6.95,7.08,7.05,7.29
Azerbaijan,4.57,4.82,4.57,4.22,4.68,4.91,5.48,5.25,5.15,5.30,5.15
Bahrain,5.70,5.70,5.70,5.94,4.82,5.03,6.69,6.17,6.01,6.17,6.23
Bangladesh,4.61,5.05,5.08,4.86,4.99,4.72,4.66,4.64,4.63,4.56,4.31


In [15]:
#create a list of indicators
indicators = new_df.keys()[2:10]
names = ['Happy_Index_mx','Log_GDP_mx','Soc_support_mx','Life_exp_mx','Freedom_mx','Generosity_mx','Corruption_mx']

print(indicators)
print(names)

Index(['Life Ladder', 'Log GDP per capita', 'Social support',
       'Healthy life expectancy at birth', 'Freedom to make life choices',
       'Generosity', 'Perceptions of corruption'],
      dtype='object')
['Happy_Index_mx', 'Log_GDP_mx', 'Soc_support_mx', 'Life_exp_mx', 'Freedom_mx', 'Generosity_mx', 'Corruption_mx']


In [16]:
#create a list of indicators
indicators = new_df.keys()[2:10]

#for all indicators create matrixes of country v/s year
for i in range (0,len(indicators)):
    df = filled_matrix_function(new_df,'Country name','year',indicators[i])
    df.to_csv(f"../Outputs/New_HI{i}_mx.csv", index=False)

# 2017 Basic Data Frame

#### After considering various factors, the group decided to evaluate the 2017 data. 
To conduct the analysis and create visualization, a dataframe with 2017 data of various variables is need. 
To do that we started with creating the a data frame that has the 128 countries that we decided to evaluate as the rows and Country Name, Code, Happiness Index and its 6 factors, as its columns.

In [17]:
H1 = HappyCountries.copy()
df_2017 = new_df[new_df["year"]==2017]
df_2017 = pd.merge(H1, df_2017, how='left', on=('Country name')).reset_index(drop=True)
df_2017.rename(columns={"Life Ladder":"Happiness Index","Log GDP per capita":"Log GDP","Healthy life expectancy at birth":"Healthy life exp",
             "Freedom to make life choices":"Freedom","Perceptions of corruption":"Pct corruption"},inplace=True)
df_2017.drop(df_2017.columns[[2]], axis = 1, inplace = True)
print(df_2017.shape) 
df_2017.head(1)         

(128, 9)


,Country name,Code,Happiness Index,Log GDP,Social support,Healthy life exp,Freedom,Generosity,Pct corruption
0,Afghanistan,AFG,2.66,7.47,0.49,52.8,0.43,-0.11,0.95


# GHDx Data

In [18]:
#Read main file - IHME-GBD_2017_DATA-ba7d53b0-1 (without age group)
GHDx_file = pd.read_csv("../Resources/IHME-GBD_2017_DATA-ba7d53b0-1.csv")            
GHDx_file.replace(['Taiwan (Province of China)'], ['Taiwan'],inplace=True)
GHDx_map = pd.read_csv("../Resources/GHDx_map.csv")

Choose appropriate data and put all together in a data frame

In [19]:
# DEATH BY SUICIDE RATE/100,000 pop MATRIX
# measure_id = 1 Death, sex_id = 3 All, CauseNo = 718 (selfharm/suicide), MetricId = 3 Rate , year = 2017
suicide_df = GHDx_file[(GHDx_file["measure_id"]==1) & (GHDx_file["sex_id"]==3) & (GHDx_file["cause_id"] == 718) & 
                  (GHDx_file["metric_id"]==3) & (GHDx_file["year"]==2017) ].round(2).reset_index()

# DEPRESSION AND ANXIETY (by Incidents) RATE/100,000 pop MATRIX 
# measure_id = 6 Incidence, sex_id = 3 All, CauseNo = 571 Anxiety disorders, year = 2017
anx_df = GHDx_file[(GHDx_file["measure_id"]==6) & (GHDx_file["sex_id"]==3) & (GHDx_file["cause_id"] == 571) & 
                  (GHDx_file["metric_id"]==3) & (GHDx_file["year"]==2017) ].round(2).reset_index()

# measure_id = 6 Incidence, sex_id = 3 All, CauseNo = 567 Depressive disorders, year = 2017
dep_df = GHDx_file[(GHDx_file["measure_id"]==6) & (GHDx_file["sex_id"]==3) & (GHDx_file["cause_id"] == 567) & 
                  (GHDx_file["metric_id"]==3) & (GHDx_file["year"]==2017) ].round(2).reset_index()

# SUBSTANCE ABUSE (by Incidents) RATE/100,000 pop MATRIX 
# measure_id = 6 Incidence, sex_id = 3 All, CauseNo = 973 Substance Abuse, year = 2017
subst_df = GHDx_file[(GHDx_file["measure_id"]==6) & (GHDx_file["sex_id"]==3) & (GHDx_file["cause_id"] == 973) & 
                  (GHDx_file["metric_id"]==3) & (GHDx_file["year"]==2017) ].round(2).reset_index()

GHDx_df = suicide_df.filter(['location_name','val'], axis=1)
GHDx_df['Depression & Anxiety'] = anx_df["val"] + dep_df["val"] 
GHDx_df['Substance Abuse'] = subst_df["val"] 
GHDx_df = GHDx_df.sort_values(by="location_name").rename(columns={"val":"Suicide"}).reset_index(drop=True)

#GHDx_df.to_csv("../Outputs/GHDx_df.csv", index=False)
print(GHDx_df.shape)
GHDx_df.head(1)

(195, 4)


,location_name,Suicide,Depression & Anxiety,Substance Abuse
0,Afghanistan,5.25,3870.85,687.47


In [20]:
mapping1 = dict(GHDx_map[['location_name', 'Code']].values)
GHDx_df.insert(1, 'Code', GHDx_map['location_name'].map(mapping1))

#### Columns are added to the df_2017 data with "left-merge" 
This method is choosen to retain it shape and to allow the value for the countries that do not exist in the variables be entered as "NaN".  
Here we add the GHDx data into the main df_2017 data frame. Afterwards we double check on the countries that has the NaN value if they are not being included just because of spelling differences.

In [21]:
df_2017 = pd.merge(df_2017 , GHDx_df, how='left', on='Code')
df_2017.drop(df_2017.columns[9], axis=1, inplace=True)
print(df_2017.shape)
df_2017.head()

(128, 12)


,Country name,Code,Happiness Index,Log GDP,Social support,Healthy life exp,Freedom,Generosity,Pct corruption,Suicide,Depression & Anxiety,Substance Abuse
0,Afghanistan,AFG,2.66,7.47,0.49,52.8,0.43,-0.11,0.95,5.25,3870.85,687.47
1,Albania,ALB,4.64,9.38,0.64,68.4,0.75,-0.03,0.88,5.86,3179.85,642.91
2,Argentina,ARG,6.04,9.85,0.91,68.6,0.83,-0.18,0.84,11.35,3666.08,1089.71
3,Armenia,ARM,4.29,9.08,0.70,66.6,0.61,-0.13,0.86,9.97,4525.35,736.38
4,Australia,AUS,7.26,10.71,0.95,73.3,0.91,0.31,0.41,12.76,3291.17,455.44


In [22]:
#check list of rows to be deleted
outcast_df1 = GHDx_df[~GHDx_df['Code'].isin(df_2017['Code'])].reset_index(drop=True)
outcast_df1.head(1)

,location_name,Code,Suicide,Depression & Anxiety,Substance Abuse
0,Algeria,DZA,4.2,4155.35,560.9


# TheGlobalEconomy Data

Most of our variable came from TheGlobalEconomy Data, which were downloaded and came as two separated csv files.
The lengthy named columns are renamed, then we picked the year and combined them into one data frame, which later be merged with the df_2017 main dataframe.
This data has the same country code, which make the data picking fairly easy.

In [23]:
Global1_file = pd.read_csv("../Resources/TheGlobalEconomy1.csv")  
Global2_file = pd.read_csv("../Resources/TheGlobalEconomy2.csv") 

In [24]:
Global1_file.rename(columns={"Gini income inequality index":"Gini index","Poverty percent of population":"Pct Poverty",
                            "Public spending on education percent of public spending":"Pct spent ed",
                             "International tourism revenue":"Rev tourism"},inplace=True)
Global2_file.rename(columns={"Fragile state index 0 (low) - 120 (high)":"Fragile index",
                             "Security threats index 0 (low) - 10 (high)":"Security index",
                             "Human flight and brain drain index 0 (low) - 10 (high)":"brain drain"},inplace=True)

In [25]:
Global1_df = Global1_file[Global1_file["Year"]==2017]
Global1_df = Global1_df.filter(['Code','Gini index','Pct Poverty','Pct spent ed','Literacy rate','Tourist arrivals','Rev tourism'], axis=1).reset_index(drop=True)  
Global_df = pd.merge(HappyCountries, Global1_df, how='left', on=('Code')).reset_index(drop=True)

Global2_df = Global2_file[Global2_file["Year"]==2017]
Global2_df = Global2_df.filter(['Code','Fragile State index','Security index','Brain drain','Unemployment rate'], axis=1).reset_index(drop=True)  
Global_df = pd.merge(Global_df, Global2_df, how='left', on=('Code')).reset_index(drop=True)

print(Global_df.shape)
Global_df.head(1)

(128, 10)


,Country name,Code,Gini index,Pct Poverty,Pct spent ed,Literacy rate,Tourist arrivals,Rev tourism,Security index,Unemployment rate
0,Afghanistan,AFG,NaN,NaN,15.66,NaN,NaN,16.0,10.0,11.18


In [26]:
df_2017 = pd.merge(df_2017, Global_df, how='left', on=('Country name','Code')).reset_index(drop=True)
print(df_2017.shape)
df_2017.head(1)

(128, 20)


,Country name,Code,Happiness Index,Log GDP,Social support,Healthy life exp,Freedom,Generosity,Pct corruption,Suicide,Depression & Anxiety,Substance Abuse,Gini index,Pct Poverty,Pct spent ed,Literacy rate,Tourist arrivals,Rev tourism,Security index,Unemployment rate
0,Afghanistan,AFG,2.66,7.47,0.49,52.8,0.43,-0.11,0.95,5.25,3870.85,687.47,NaN,NaN,15.66,NaN,NaN,16.0,10.0,11.18


# Other Files

We added other datas that is being evaluated such as GDP, number of Physician perpop, Average Age, and sleep per (min) 

In [27]:
# GDP PPP PerPopulation
gdp_df = pd.read_csv("../Resources/gdp_ppp_pop.csv", encoding='cp1252')
gdp_df = gdp_df.filter(["Country Name","Country Code","2017"], axis=1).round(2)

gdp_df = pd.merge(HappyCountries, gdp_df, how='left', left_on='Code', right_on='Country Code',
         left_index=False, right_index=False, sort=True)

df_2017.insert(2,('GDP'),gdp_df['2017'])

print(df_2017.shape)
df_2017.head(1)

(128, 21)


,Country name,Code,GDP,Happiness Index,Log GDP,Social support,Healthy life exp,Freedom,Generosity,Pct corruption,...,Depression & Anxiety,Substance Abuse,Gini index,Pct Poverty,Pct spent ed,Literacy rate,Tourist arrivals,Rev tourism,Security index,Unemployment rate
0,Afghanistan,AFG,2202.57,2.66,7.47,0.49,52.8,0.43,-0.11,0.95,...,3870.85,687.47,NaN,NaN,15.66,NaN,NaN,16.0,10.0,11.18


In [28]:
# Physician perpop
#url'https://data.worldbank.org/indicator/SH.MED.PHYS.ZS'
df_hcd=pd.read_csv('../Resources/doctors_population_WB1-2019.csv')
df_hcd = df_hcd.filter(["Country Name","Country Code","2017"], axis=1).round(2)
df_hcd = pd.merge(HappyCountries, df_hcd, how='left', left_on='Code', right_on='Country Code')

df_2017.insert(13,('Physician perpop'),df_hcd['2017'])

print(df_2017.shape)
df_2017.head(1)

(128, 22)


,Country name,Code,GDP,Happiness Index,Log GDP,Social support,Healthy life exp,Freedom,Generosity,Pct corruption,...,Substance Abuse,Physician perpop,Gini index,Pct Poverty,Pct spent ed,Literacy rate,Tourist arrivals,Rev tourism,Security index,Unemployment rate
0,Afghanistan,AFG,2202.57,2.66,7.47,0.49,52.8,0.43,-0.11,0.95,...,687.47,NaN,NaN,NaN,15.66,NaN,NaN,16.0,10.0,11.18


In [29]:
# Average Age
Age_file = pd.read_csv("../Resources/Avg.age2017-final.csv") 

del Age_file['Column1']
Age_file.rename(columns={ Age_file.columns[0]: "Country Name", Age_file.columns[1]:"Average age" }, inplace = True)
Age_file['Country Name'] = Age_file['Country Name'].str.strip()

Age_df = pd.merge(HappyCountries, Age_file, how='left', left_on='Country name',  #fit data to standard dataframe
         right_on='Country Name',left_index=False, right_index=False, sort=True)

df_2017.insert(14,('Average Age'),Age_df['Average age'])                        #add to df_2017 dataframe
print(df_2017.shape)
df_2017.head(1)

(128, 23)


,Country name,Code,GDP,Happiness Index,Log GDP,Social support,Healthy life exp,Freedom,Generosity,Pct corruption,...,Physician perpop,Average Age,Gini index,Pct Poverty,Pct spent ed,Literacy rate,Tourist arrivals,Rev tourism,Security index,Unemployment rate
0,Afghanistan,AFG,2202.57,2.66,7.47,0.49,52.8,0.43,-0.11,0.95,...,NaN,18.8,NaN,NaN,15.66,NaN,NaN,16.0,10.0,11.18


In [30]:
# Sleep
Sleep_file = pd.read_csv("../Resources/Avg_sleep_minutes.csv") 

sleep_df = pd.merge(HappyCountries, Sleep_file, how='left', left_on='Country name',  #fit data to standard dataframe
         right_on='Country',left_index=False, right_index=False, sort=True)

df_2017.insert(15,('Sleep (min)'),sleep_df['Minutes'])                        #add to df_2017 dataframe
print(df_2017.shape)
df_2017.head(1)

(128, 24)


,Country name,Code,GDP,Happiness Index,Log GDP,Social support,Healthy life exp,Freedom,Generosity,Pct corruption,...,Average Age,Sleep (min),Gini index,Pct Poverty,Pct spent ed,Literacy rate,Tourist arrivals,Rev tourism,Security index,Unemployment rate
0,Afghanistan,AFG,2202.57,2.66,7.47,0.49,52.8,0.43,-0.11,0.95,...,18.8,NaN,NaN,NaN,15.66,NaN,NaN,16.0,10.0,11.18


In [31]:
df_2017.to_csv("../Outputs/df_2017.csv", index=True)

# Normalisation

To be able to compare various variables, their values are projected into 0 - 100 scale. 
We determine the max and minimum of each variable, then calculate their ranges. Those are then beig used to normalize the value.

In [32]:
min_value = df_2017.min()
max_value = df_2017.max()

norml_df = pd.DataFrame({'Min Value':min_value,'Max Value':max_value})
norml_df.drop(norml_df.index[:3], inplace=True)
norml_df['range'] = norml_df['Max Value'] - norml_df['Min Value']

norml_df.to_csv("../Outputs/Normalization value 2017.csv", index=True)
norml_df

,Min Value,Max Value,range
Happiness Index,2.66,7.79,5.13
Log GDP,6.83,11.44,4.61
Social support,0.44,0.96,0.52
Healthy life exp,47.7,76.5,28.8
Freedom,0.43,0.99,0.56
Generosity,-0.29,0.48,0.77
Pct corruption,0.16,0.95,0.79
Suicide,2.86,57.61,54.75
Depression & Anxiety,1843.64,7022.08,5178.44
Substance Abuse,296.64,2282.46,1985.82


In [33]:
col_2017 = df_2017.keys()
print(col_2017)

Index(['Country name', 'Code', 'GDP', 'Happiness Index', 'Log GDP',
       'Social support', 'Healthy life exp', 'Freedom', 'Generosity',
       'Pct corruption', 'Suicide', 'Depression & Anxiety', 'Substance Abuse',
       'Physician perpop', 'Average Age', 'Sleep (min)', 'Gini index',
       'Pct Poverty', 'Pct spent ed', 'Literacy rate', 'Tourist arrivals',
       'Rev tourism', 'Security index', 'Unemployment rate'],
      dtype='object')


In [34]:
#NORMALIZATION TABLE 2017
norm_2017_df = country_code.copy()   
for i in range(3,len(col_2017)):
    norm_2017_df[col_2017 [i]]= (df_2017[col_2017 [i]] - norml_df.loc[col_2017 [i],'Min Value']) / norml_df.loc[col_2017 [i],'range']

norm_2017_df['Blue Index'] = norm_2017_df[['Suicide','Depression & Anxiety','Substance Abuse']].mean(axis=1)
norm_2017_df.to_csv("../Outputs/Normalized Table 2017_df.csv", index=False)
norm_2017_df.round(2)

,Country name,Code,Continent,Happiness Index,Log GDP,Social support,Healthy life exp,Freedom,Generosity,Pct corruption,...,Sleep (min),Gini index,Pct Poverty,Pct spent ed,Literacy rate,Tourist arrivals,Rev tourism,Security index,Unemployment rate,Blue Index
0,Afghanistan,AFG,Asia,0.00,0.14,0.10,0.18,0.00,0.23,1.00,...,NaN,NaN,NaN,0.50,NaN,NaN,0.00,1.00,0.40,0.21
1,Albania,ALB,Europe,0.39,0.55,0.38,0.72,0.57,0.34,0.91,...,NaN,NaN,NaN,0.31,NaN,0.05,0.01,0.41,0.50,0.16
2,Argentina,ARG,Americas,0.66,0.66,0.90,0.73,0.71,0.14,0.86,...,0.51,0.54,0.34,0.36,NaN,0.08,0.02,0.37,0.30,0.30
3,Armenia,ARM,Asia,0.32,0.49,0.50,0.66,0.32,0.21,0.89,...,NaN,0.29,0.34,0.20,1.00,0.02,0.00,0.50,0.65,0.29
4,Australia,AUS,Oceania,0.90,0.84,0.98,0.89,0.86,0.78,0.32,...,0.94,NaN,NaN,NaN,NaN,0.10,0.17,0.12,0.19,0.18
5,Austria,AUT,Europe,0.90,0.85,0.90,0.87,0.82,0.56,0.46,...,0.79,NaN,NaN,NaN,NaN,0.34,0.09,0.00,0.19,0.53
6,Azerbaijan,AZE,Asia,0.49,0.62,0.67,0.61,0.54,0.06,0.62,...,NaN,NaN,NaN,0.00,1.00,0.03,0.01,0.58,0.17,0.24
7,Bahrain,BHR,Asia,0.70,0.84,0.85,0.72,0.86,0.55,NaN,...,NaN,NaN,NaN,0.01,0.95,0.13,0.02,0.66,0.01,0.34
8,Bangladesh,BGD,Asia,0.32,0.30,0.52,0.56,0.84,0.40,0.61,...,NaN,NaN,NaN,0.66,0.48,0.01,0.00,0.73,0.15,0.11
9,Belarus,BLR,Europe,0.56,0.63,0.88,0.63,0.34,0.21,0.62,...,NaN,0.00,0.05,0.31,NaN,0.13,0.00,0.57,0.19,0.27


# Adding continents

For visualization purposes we added continents to our normalized dataframe

In [35]:
norm_2017_v2_df = norm_2017_df.copy()
mapping1 = dict(country_code[['Code','Continent']].values)
norm_2017_v2_df['Continent'] = country_code['Code'].map(mapping1)

norm_2017_v2_df.to_csv("../Outputs/Normalized Table 2017_df-v2.csv", index=False)

print(norm_2017_v2_df.shape)
norm_2017_v2_df.head(1)

(128, 25)


,Country name,Code,Continent,Happiness Index,Log GDP,Social support,Healthy life exp,Freedom,Generosity,Pct corruption,...,Sleep (min),Gini index,Pct Poverty,Pct spent ed,Literacy rate,Tourist arrivals,Rev tourism,Security index,Unemployment rate,Blue Index
0,Afghanistan,AFG,Asia,0.0,0.138829,0.096154,0.177083,0.0,0.233766,1.0,...,NaN,NaN,NaN,0.49914,NaN,NaN,0.0,1.0,0.402182,0.210645
